In [ ]:
# import packages
import pandas as pd
import geopandas as gpd
import geojson
import numpy as np
import os
from shapely.geometry import Point

# Import Data

In [ ]:
# set location of input data files
input_dir = 'input_data'

### Segment polylines

In [ ]:
# read in segment shapefile (simplified to 4 percent with mapshaper and smoothed in ArcGIS) as geodataframe
segment_gdf = gpd.read_file(os.path.join(input_dir,'shapefiles/Segments_subset_4per_smooth.shp'))

### Delaware Bay

In [ ]:
# read in NHD delaware bay shapefile (simplified to 0.6 percent with mapshaper and smoothed in ArcGIS) as geodataframe
delaware_bay_gdf = gpd.read_file(os.path.join(input_dir,'shapefiles/NHDWaterbody_DelawareBay_pt6per_smooth.shp'))

### Reservoirs

In [ ]:
# read in simplified reservoirs shapefile as geodataframe
reservoirs = gpd.read_file(os.path.join(input_dir,'shapefiles/reservoirs_17per.shp'))

In [ ]:
# filter out four reservoirs that are not on model segments
reservoirs = reservoirs[(reservoirs.GRAND_ID != 2212) & (reservoirs.GRAND_ID != 1591) & (reservoirs.GRAND_ID != 1584) & (reservoirs.GRAND_ID != 2242)]

In [ ]:
# drop index
reservoirs = reservoirs.reset_index(drop=True)

### Station locations

In [ ]:
# list of unique drb sites (coordinate system = NAD1983 = EPSG 4269)
unique_drb_sites = pd.read_csv(os.path.join(input_dir,'unique_drb_sites.csv'), index_col=None)

### Temperature observations

In [ ]:
obs_temp_df_raw = pd.read_csv(os.path.join(input_dir,'obs_temp_drb.csv'), delimiter=',')

### # Observations per year, by source

In [ ]:
n_obs_annual_source = pd.read_csv(os.path.join(input_dir,'n_obs_per_year_source.csv'), delimiter=',')

### Modeled segment outflow

In [ ]:
seg_outflow = pd.read_csv(os.path.join(input_dir,'seg_outflow.csv'))

# Clean and structure data for analysis

### Clean temperature observation data

In [ ]:
# remove NAs from seg_id_nat column
obs_temp_df_cleaned = obs_temp_df_raw.loc[obs_temp_df_raw['seg_id_nat'].notnull()]

In [ ]:
# get # of unique segment ids
obs_temp_seg_id_nat_unique = np.unique(obs_temp_df_cleaned['seg_id_nat'].tolist())
len(obs_temp_seg_id_nat_unique)

In [ ]:
# convert date to datetime
obs_temp_df_cleaned['date'] = pd.to_datetime(obs_temp_df_cleaned['date'])

In [ ]:
# find single erroneous observation value
obs_temp_df_cleaned.loc[(obs_temp_df_cleaned['date'] == '2019-08-21') & (obs_temp_df_cleaned['seg_id_nat'] == 1764)]

In [ ]:
# drop that row from the dataframe
obs_temp_df_cleaned = obs_temp_df_cleaned.drop(labels=307015)

In [ ]:
# Set date as index and sort by date
obs_temp_df_cleaned = obs_temp_df_cleaned.set_index(['date'], drop=True)
obs_temp_df_cleaned = obs_temp_df_cleaned.sort_index()

In [ ]:
# create new copy of cleaned dataframe
obs_temp_df = obs_temp_df_cleaned.copy()

In [ ]:
# convert segment id to integer
obs_temp_df['seg_id_nat'] = obs_temp_df['seg_id_nat'].astype(int)
# create year column based on year index
obs_temp_df['year'] = obs_temp_df.index.year #.astype(int)
# create year-month column
obs_temp_df['year-month'] = obs_temp_df.index.to_period('M')
# create month column
obs_temp_df['month'] = obs_temp_df.index.month
# for each row (i.e. each observation), set observation count to 1
obs_temp_df['obs_count'] = 1
obs_temp_df.head()

In [ ]:
# filter observations data to only include observations from 1980-2020
obs_temp_df = obs_temp_df.loc['1980-01-01':'2020-12-31']
obs_temp_df

### Generate date list and dataframe for use later to pull segment-specific data

In [ ]:
# create version of observations dataframe with date (as string) and segment id as indices
obs_temp_daily_count = obs_temp_df.copy()
obs_temp_daily_count.index = obs_temp_daily_count.index.astype(str)
obs_temp_daily_count

In [ ]:
# add segment id as second index level (for use later to pull observations for each segment)
obs_temp_daily_count = obs_temp_daily_count.set_index('seg_id_nat', append=True)
obs_temp_daily_count.head()

# Process spatial data

### Get list of all segments

In [ ]:
segment_list = np.unique(segment_gdf['seg_id_nat'].tolist())
segment_list.sort()

### Get centroids for stream segments

In [ ]:
# create copy of dataframe
centroid_gdf = segment_gdf.copy()

In [ ]:
# check crs of segment geodataframe
centroid_gdf.crs

In [ ]:
# reproject before calculating segment centroids
centroid_gdf = centroid_gdf.to_crs(epsg=6350)

In [ ]:
# check crs of segment geodataframe
centroid_gdf.crs

In [ ]:
# add column with centroid of each segment
centroid_gdf['centroid'] = centroid_gdf['geometry'].interpolate(0.5, normalized=True)
centroid_gdf.head()

In [ ]:
# reassign the geodataframe geometry to be the centroid column
centroid_gdf = centroid_gdf.set_geometry('centroid')

In [ ]:
# revert to geographic crs (WGS 1984)
centroid_gdf = centroid_gdf.to_crs(epsg=4326)

In [ ]:
# check that centroid coordinates have been reprojected
centroid_gdf.head()

In [ ]:
# create column of only centroid latitude
centroid_gdf['seg_centroid_lat'] = centroid_gdf['centroid'].apply(lambda p: p.y)
centroid_gdf.head()

### Extract latitudes of segment centroids

In [ ]:
# drop all columns except seg_id_nat and centroid
segment_latitudes_df = centroid_gdf.drop(columns=['region','model_idx','InLine_FID','SmoLnFlag','geometry', 'centroid'])

In [ ]:
# set segment id as index
segment_latitudes_df = segment_latitudes_df.set_index('seg_id_nat')
segment_latitudes_df.head()

### Convert dataframe of unique sites in DRB with monitoring data to geodataframe

In [ ]:
unique_drb_sites.head()

In [ ]:
# convert the dataframe to a geodataframe
unique_drb_sites_gdf = gpd.GeoDataFrame(unique_drb_sites, crs="EPSG:4326", geometry=gpd.points_from_xy(unique_drb_sites.longitude, unique_drb_sites.latitude))
unique_drb_sites_gdf.head()

In [ ]:
unique_drb_sites_gdf.shape

In [ ]:
ax = segment_gdf.plot(figsize=(30,15))
unique_drb_sites_gdf.plot(ax = ax, markersize=2, color = 'red')

# Process temperature observations data

### Transform dataframe of # of observations per year per source

In [ ]:
# load data w/ number of observations per year, by source
n_obs_annual_source.head()

In [ ]:
# pivot into wide format
source_annual_count = n_obs_annual_source.pivot(index='year', columns='source', values='n_obs')
source_annual_count.head()

In [ ]:
# create a continous date range from 1901-2020, at an annual interval
year_index = pd.date_range('01-01-1960', '01-01-2021', freq='A')

In [ ]:
# pull only the year associated with each date
year_index = year_index.year

In [ ]:
# reindex the dataframe to fill in missing years
# fill nas with 0s
source_annual_count = source_annual_count.reindex(index=year_index, fill_value = 0)
source_annual_count = source_annual_count.fillna(value = 0)
source_annual_count = source_annual_count.astype(int)

In [ ]:
source_annual_count.index.rename('year', inplace=True)
source_annual_count = source_annual_count.rename(columns={'Other':'State or other agency'})
source_annual_count = source_annual_count[['USGS', 'State or other agency']]
source_annual_count

### Get segment-specific counts of temperature observations for different time steps

##### count of all observations from 1980-2020 for each segment

In [ ]:
obs_temp_df.head()

In [ ]:
# count of all observations from 1980-2020 for each segment
obs_temp_count = obs_temp_df.groupby(['seg_id_nat']).count()
obs_temp_count = obs_temp_count.drop(columns=['subseg_id', 'mean_temp_c', 'min_temp_c', 'max_temp_c', 'site_id', 'year', 'year-month', 'month'])
obs_temp_count = obs_temp_count.sort_index()
obs_temp_count.index = obs_temp_count.index.astype(int)
obs_temp_count = obs_temp_count.rename(columns={'obs_count':'total_count'})
obs_temp_count

##### count of observations in each year (from 1980-2020) for each segment

In [ ]:
# count of observations in each year, from 1980 - 2020
obs_temp_year_count = obs_temp_df.groupby(['seg_id_nat', 'year']).count()
obs_temp_year_count = obs_temp_year_count.drop(columns=['subseg_id', 'mean_temp_c', 'min_temp_c', 'max_temp_c', 'site_id', 'year-month', 'month'])

##### count of all observations in 2019 for each segment

In [ ]:
# subset 2019 data from all temperature observations
obs_temp_df_2019 = obs_temp_df.loc['2019-01-01':'2019-12-31']
obs_temp_df_2019

In [ ]:
# count of all observations in each month of 2019 for each segment
obs_temp_df_2019_months = obs_temp_df_2019.copy()
obs_temp_df_2019_months['month_name'] = obs_temp_df_2019_months.index.strftime('%B')
obs_temp_count_month_2019 = obs_temp_df_2019_months.groupby(['seg_id_nat','month_name']).count()
obs_temp_count_month_2019 = obs_temp_count_month_2019.drop(columns=['subseg_id','mean_temp_c','min_temp_c','max_temp_c','site_id','year','year-month','month'])
obs_temp_mean_month_2019 = obs_temp_df_2019_months.groupby(['seg_id_nat','month_name']).mean()
obs_temp_mean_month_2019 = obs_temp_mean_month_2019.drop(columns=['min_temp_c','max_temp_c','year','month','obs_count'])
obs_temp_mean_month_2019 = obs_temp_mean_month_2019.rename(columns={'mean_temp_c':'mean_t_c'})
obs_temp_month_2019 = obs_temp_count_month_2019.join(obs_temp_mean_month_2019)

In [ ]:
obs_temp_month_2019.head()

In [ ]:
# count of all observations on each day in 2019 for each segment
obs_temp_count_2019 = obs_temp_df_2019.groupby(['seg_id_nat']).count()
obs_temp_count_2019 = obs_temp_count_2019.drop(columns=['month', 'year', 'subseg_id', 'mean_temp_c', 'min_temp_c', 'max_temp_c', 'site_id', 'year-month'])
obs_temp_count_2019 = obs_temp_count_2019.sort_index()
obs_temp_count_2019.index = obs_temp_count_2019.index.astype(int)
obs_temp_count_2019 = obs_temp_count_2019.rename(columns={'obs_count':'total_count'})
obs_temp_count_2019

### get total annual and daily counts of observations

##### annual counts

In [ ]:
# group observations by year to get total count of observations in each year
obs_annual_count = obs_temp_df.groupby(['year']).count()
obs_annual_count = obs_annual_count.drop(columns=['subseg_id', 'seg_id_nat', 'mean_temp_c', 'min_temp_c', 'max_temp_c', 'site_id', 'year-month', 'month'])
obs_annual_count = obs_annual_count.rename(columns={'obs_count': 'total_annual_count'})
obs_annual_count.head()

##### daily counts

In [ ]:
# get count of observations on each day from 1980-2019
obs_daily_count = obs_temp_df.groupby(['date']).count()
obs_daily_count = obs_daily_count.drop(columns=['subseg_id', 'seg_id_nat', 'mean_temp_c', 'min_temp_c', 'max_temp_c', 'site_id', 'year', 'year-month', 'month'])
obs_daily_count = obs_daily_count.rename(columns={'obs_count': 'total_daily_count'})
obs_daily_count

In [ ]:
# get count of observations on each day of 2019
obs_daily_count_2019 = obs_daily_count.loc['2019-01-01':'2019-12-31']
obs_daily_count_2019

### Format data for matrices

In [ ]:
# create copy of observations dataframe
all_observations_df = obs_temp_df.copy()
# set year as string type
all_observations_df['year'] = all_observations_df['year'].astype(str)
all_observations_df

##### annual time interval

In [ ]:
# create a dataframe with the segment ids as the columns
matrix_annual_df = pd.DataFrame(columns=segment_list)
# set a date range with annual timesteps from 1980-2020
model_date_rng = pd.date_range('1980', periods=41, freq='A')
# add a column to the dataframe with the set date range
matrix_annual_df['Date'] = model_date_rng
# convert dates to datetime format
matrix_annual_df['Date'] = pd.to_datetime(matrix_annual_df['Date'])
# set date as index
matrix_annual_df = matrix_annual_df.set_index('Date')
# create a column for year
matrix_annual_df['year'] = matrix_annual_df.index.to_period('A')
# set year as index
matrix_annual_df = matrix_annual_df.set_index('year')
# make index type string
matrix_annual_df.index = matrix_annual_df.index.astype(str)
matrix_annual_df.head()

In [ ]:
# stack the dataframe columns to indices
matrix_annual_series = matrix_annual_df.stack(dropna=False)
matrix_annual_series

In [ ]:
# convert the stacked series to a dataframe with two indices
matrix_annual_stacked = matrix_annual_series.to_frame()
# rename the second index to segment id
matrix_annual_stacked.index = matrix_annual_stacked.index.rename('seg_id_nat', level=1)
matrix_annual_stacked.head()

In [ ]:
# get count of observations for each segment in each year
seg_obs_temp_year_count = all_observations_df.groupby(['year','seg_id_nat']).sum()
seg_obs_temp_year_count = seg_obs_temp_year_count.drop(columns=['mean_temp_c','min_temp_c','max_temp_c','month'])
seg_obs_temp_year_count.head()

In [ ]:
# add the count for each segment to the matrix
matrix_annual_obs = matrix_annual_stacked.join(seg_obs_temp_year_count, on=['year','seg_id_nat'])
# drop empty column
matrix_annual_obs = matrix_annual_obs.drop(columns=0)
# replace na values with 0 (for 0 observations)
matrix_annual_obs = matrix_annual_obs.fillna(0, axis=0)
matrix_annual_obs.head()

In [ ]:
# add column with total count for each segment (over whole period, from 1980-2020)
matrix_annual_obs = matrix_annual_obs.join(obs_temp_count, on=['seg_id_nat'], how='left')
matrix_annual_obs = matrix_annual_obs.fillna(0)
matrix_annual_obs.head()

In [ ]:
# create dataframe of segment latitudes, ordered by latitude
segment_latitudes_reindexed = segment_latitudes_df.sort_values(by='seg_centroid_lat')
# reset the index
segment_latitudes_reindexed = segment_latitudes_reindexed.reset_index()
# set the segment id as the second index
segment_latitudes_reindexed = segment_latitudes_reindexed.set_index('seg_id_nat', append=True)
segment_latitudes_reindexed.head()

In [ ]:
# make a column storing the first and second index levels as a tuple
segment_latitudes_reindexed['index_tuple'] = segment_latitudes_reindexed.index
# drop the first level of the index
segment_latitudes_reindexed = segment_latitudes_reindexed.droplevel(level=0)
segment_latitudes_reindexed.head()

In [ ]:
# add column with zero values named 'rank'
segment_latitudes_reindexed['rank'] = 0
segment_latitudes_reindexed.head()

In [ ]:
# fill the rank column with the first value of the index_tuple (to get rank of segment by latitude)
for segment_id in segment_list:
    segment_latitudes_reindexed['rank'][segment_id] = segment_latitudes_reindexed['index_tuple'][segment_id][0]
segment_latitudes_reindexed.head()

In [ ]:
# sort the dataframe by the segment id
segment_latitudes_reindexed = segment_latitudes_reindexed.sort_index()
# drop the index tuple column
segment_latitudes_reindexed = segment_latitudes_reindexed.drop(columns=['index_tuple'])
segment_latitudes_reindexed.head()

In [ ]:
# join the segment latitudes dataframe to the matrix of segment observations
matrix_annual_obs = matrix_annual_obs.join(segment_latitudes_reindexed, on=['seg_id_nat'], how='left')
matrix_annual_obs.head()

In [ ]:
# sort the matrix by rank, so that the segment data is ordered by segment latitude
matrix_annual_obs = matrix_annual_obs.sort_values(by='rank')
# sort the matrix by year, so that data is ordered correctly temporally
matrix_annual_obs = matrix_annual_obs.sort_index(level=0, sort_remaining=False)
matrix_annual_obs.head()

##### daily time interval - 2019 only

In [ ]:
# create an empty dataframe with the segment ids as the columns
matrix_daily_2019_df = pd.DataFrame(columns=segment_list)
# set up a date range with a daily timestep for the year 2019
model_daily_2019_date_rng = pd.date_range('2019-01-01', periods=365, freq='D')
# add a date column to the dataframe based on the date range
matrix_daily_2019_df['date'] = model_daily_2019_date_rng
# convert the date to datetime format
matrix_daily_2019_df['date'] = pd.to_datetime(matrix_daily_2019_df['date'])
# set the date as the index
matrix_daily_2019_df = matrix_daily_2019_df.set_index('date')
matrix_daily_2019_df.head()

In [ ]:
# stack the dataframe columns to indices
matrix_daily_2019_series = matrix_daily_2019_df.stack(dropna=False)
matrix_daily_2019_series

In [ ]:
# create a dataframe with two index levels from the stacked series
matrix_daily_2019_stacked = matrix_daily_2019_series.to_frame()
# rename the second index level 'seg_id_nat'
matrix_daily_2019_stacked.index = matrix_daily_2019_stacked.index.rename('seg_id_nat', level=1)
matrix_daily_2019_stacked.head()

In [ ]:
# get count of observations for each segment on each day
seg_obs_temp_daily_count = all_observations_df.groupby(['date','seg_id_nat']).sum()
seg_obs_temp_daily_count = seg_obs_temp_daily_count.drop(columns=['mean_temp_c','min_temp_c','max_temp_c','month'])
seg_obs_temp_daily_count.head()

In [ ]:
# join the daily counts to the observation matrix
matrix_daily_2019_obs = matrix_daily_2019_stacked.join(seg_obs_temp_daily_count, on=['date','seg_id_nat'])
# drop empty column
matrix_daily_2019_obs = matrix_daily_2019_obs.drop(columns=0)
# replace na values with 0s
matrix_daily_2019_obs = matrix_daily_2019_obs.fillna(0, axis=0)
matrix_daily_2019_obs.head()

In [ ]:
# add a column with the total count for each segment (in 2019)
matrix_daily_2019_obs = matrix_daily_2019_obs.join(obs_temp_count_2019, on=['seg_id_nat'], how='left')
matrix_daily_2019_obs = matrix_daily_2019_obs.fillna(0)
matrix_daily_2019_obs.head()

In [ ]:
# pull actual temperature observations
obs_temp_daily_count_temps = all_observations_df.copy()
obs_temp_daily_count_temps = obs_temp_daily_count_temps.set_index('seg_id_nat', append=True)
obs_temp_daily_count_temps = obs_temp_daily_count_temps.drop(columns=['subseg_id','min_temp_c','max_temp_c','site_id','year','year-month','month','obs_count'])
obs_temp_daily_count_temps.head()

In [ ]:
# add in temperature for each date for each segment
matrix_daily_2019_obs = matrix_daily_2019_obs.join(obs_temp_daily_count_temps, on=['date','seg_id_nat'])
matrix_daily_2019_obs.head()

In [ ]:
# add latitude and latitude-based rank of each segment
matrix_daily_2019_obs = matrix_daily_2019_obs.join(segment_latitudes_reindexed, on=['seg_id_nat'], how='left')
matrix_daily_2019_obs.head()

In [ ]:
# sort dataframe by rank
matrix_daily_2019_obs = matrix_daily_2019_obs.sort_values(by='rank')
# sort dataframe by date
matrix_daily_2019_obs = matrix_daily_2019_obs.sort_index(level=0, sort_remaining=False)
matrix_daily_2019_obs.head()

# Processed modeled segment outflow data

In [ ]:
seg_outflow.head()

In [ ]:
# convert date to datetime format
seg_outflow['date'] = pd.to_datetime(seg_outflow['date'])

In [ ]:
# set date as index
seg_outflow = seg_outflow.set_index(['date'], drop=True)

In [ ]:
# add year column
seg_outflow['year'] = seg_outflow.index.year

In [ ]:
# subset the data to a 30yr period
seg_outflow_81_09 = seg_outflow.loc[(seg_outflow['year'] > 1980) & (seg_outflow['year'] < 2010)]

In [ ]:
# compute average modeled outflow for each segment in each year
seg_avg_outflow_81_09 = seg_outflow_81_09.groupby(['year']).mean()
seg_avg_outflow_81_09.head()

In [ ]:
# transform the dataframe, so that the segment id is the index
segindex_avg_outflow_81_09 = seg_avg_outflow_81_09.T

In [ ]:
# add column with segment id
segindex_avg_outflow_81_09['seg_id_nat'] = segindex_avg_outflow_81_09.index

In [ ]:
# convert segment id to integer type
segindex_avg_outflow_81_09['seg_id_nat'] = segindex_avg_outflow_81_09['seg_id_nat'].astype(int)

In [ ]:
# add a column with the overall average annual modeled outflow for each segment
segindex_avg_outflow_81_09['avg_ann_flow'] = segindex_avg_outflow_81_09.mean(axis=1)
segindex_avg_outflow_81_09.head()

In [ ]:
# subset data to just segment id and average annual modeled segment outflow
segment_maflow = segindex_avg_outflow_81_09[['seg_id_nat','avg_ann_flow']]
segment_maflow = segment_maflow.set_index('seg_id_nat')
segment_maflow.head()

# Export data

In [ ]:
# set location for intermediate output data files
intermediate_output_dir = 'intermediate_output'

In [ ]:
# create intermediate output folder if it doesn't already exist
if not os.path.exists(intermediate_output_dir):
    os.mkdir(intermediate_output_dir)

In [ ]:
# set location for final output data files
output_dir = '../public/data'

### Export spatial data that does not require processing

##### Generate reservoir geojson

In [ ]:
# export geodataframe as a geojson
reservoirs.to_file(os.path.join(intermediate_output_dir,'reservoirs.json'), driver='GeoJSON')

##### Generate delaware bay geojson

In [ ]:
delaware_bay_gdf.to_file(os.path.join(intermediate_output_dir,'NHDWaterbody_DelawareBay_pt6per_smooth.json'), driver='GeoJSON')

### Export processed datasets

##### locations of unique monitoring sites with temperature observations

In [ ]:
# convert geodataframe to geojson
unique_drb_sites_gdf.to_file(os.path.join(intermediate_output_dir,'unique_drb_sites.json'), driver='GeoJSON')

##### count of observations at sites associated with each agency in each year

In [ ]:
source_annual_count.to_csv(os.path.join(output_dir,'source_annual_count.csv'), index_label=None)

##### temporal counts of temperature observations (not segment-specific)

In [ ]:
# export total counts for each year from 1980-2019
obs_annual_count.to_csv(os.path.join(output_dir,'obs_annual_count.csv'), index_label=None)

In [ ]:
# export total counts for each day of 2019
obs_daily_count_2019.to_csv(os.path.join(output_dir,'obs_daily_count_2019.csv'), index_label=None)

##### matrix of segment temperature observations on annual timestep

In [ ]:
matrix_annual_obs.to_csv(os.path.join(output_dir,'matrix_annual_obs.csv'), index_label=None)

##### matrix of segment temperature observations on daily timestep (2019 only)

In [ ]:
matrix_daily_2019_obs.to_csv(os.path.join(output_dir,'matrix_daily_2019_obs.csv'), index_label=None)

##### mean annual modeled segment outflow for each model segment

In [ ]:
# segment_maflow.to_csv('TP_output_data/segment_maflow.csv', index_label='seg_id_nat')
segment_maflow.to_csv(os.path.join(output_dir,'segment_maflow.csv'), index_label=None)

### Construct and export segment geojson

##### prep data

In [ ]:
# convert segment geodataframe to dictionary format
segment_polylines = segment_gdf.to_dict(orient='records')
len(segment_polylines)

In [ ]:
# get list of years in record (1980-2014)
year_list = np.unique(obs_temp_df['year'].tolist())
year_list.sort()

In [ ]:
month_list = ['January','February','March','April','May','June','July','August','September','October','November','December']

##### construct json

In [ ]:
# format designed to match desired structure of segmentDict
# create empty array to store dictionaries
segment_array = []
# iterate through the list of segments to...
i = 0
while i < len(segment_polylines):
    # create an empty segment dictionary
    segment_dict = {}
    # set type to Feature
    segment_dict["type"] = "Feature"
    # set segment id field
    segment_id = segment_polylines[i]['seg_id_nat']
    segment_dict["seg_id_nat"] = str(segment_id)
    # add properties dictionary
    segment_dict["properties"] = {}
    
    # Add segment id as outer key
    segment_dict["properties"]["seg_id_nat"] = str(segment_id)
    
    # Add segment id as outer key
    segment_dict["properties"][str(segment_id)] = {}
    
    # add average annual flow
    segment_dict["properties"][str(segment_id)]['avg_ann_flow'] = str(segment_maflow['avg_ann_flow'][segment_id])
      
    # Add total count of observations in each segment
    try:
        segment_dict["properties"][str(segment_id)]["total_count"] = str(obs_temp_count['total_count'][segment_id])
    except:
        segment_dict["properties"][str(segment_id)]["total_count"] = '0'
    
    # create dictionary to store count for each year on record
    segment_dict["properties"][str(segment_id)]["year_count"] = {}
    # iterate through years in list of years to...
    for year in year_list:
        # add the count of observations for each segment in that year
        try:
            segment_dict["properties"][str(segment_id)]["year_count"][str(year)] = str(obs_temp_year_count['obs_count'][segment_id][year])
        except:
            segment_dict["properties"][str(segment_id)]["year_count"][str(year)] = '0'
            
    # add dictionary to store monthly data from 2019 for each segment
    segment_dict["properties"][str(segment_id)]["data_2019_monthly"] = {}
    for month in month_list:
        try:
            obs_temp_month_2019['obs_count'][segment_id][month]
            segment_dict["properties"][str(segment_id)]["data_2019_monthly"][month] = {}
            segment_dict["properties"][str(segment_id)]["data_2019_monthly"][month]["month_count"] = str(obs_temp_month_2019['obs_count'][segment_id][month])
            segment_dict["properties"][str(segment_id)]["data_2019_monthly"][month]["month_avg_temp"] = str(obs_temp_month_2019['mean_t_c'][segment_id][month])
        except:
            continue
    
    # add geometry based on segment geometry
    segment_dict["geometry"] = segment_polylines[i]["geometry"]
    
    # append the segment dictionary to the empty segment array
    segment_array.append(segment_dict)
    
    # print statement indicating progress
    print("added segment", str(i+1), "of", len(segment_polylines))
    
    # increase counter for loop
    i += 1

In [ ]:
# create empty feature collection dictionary
segment_feature_collection = {}
segment_feature_collection["type"] = "FeatureCollection"
# set content of segment array as list of features within feature collection
segment_feature_collection["features"] = segment_array

In [ ]:
# convert segment feature collection to json format
segment_geojson = geojson.dumps(segment_feature_collection)

In [ ]:
# export formatted segment geojson
with open(os.path.join(intermediate_output_dir,'segment_data.json'), 'w', encoding='utf-8') as json_file:
    json_file.write(segment_geojson)

# Convert exported geojsons to topojsons

##### Requires installation of mapshaper: https://github.com/mbloch/mapshaper

In [ ]:
# get documentation
! mapshaper -h

In [ ]:
# convert reservoir json in intermediate_output folder to topojson w/ reduced precision and save in topojson subfolder of public data folder
! mapshaper -i intermediate_output/reservoirs.json -o ../public/data/topojson/reservoirs.json format=topojson precision=0.001

In [ ]:
# convert unique_drb_sites json in intermediate_output folder to topojson w/ reduced precision and save in topojson subfolder of public data folder
! mapshaper -i intermediate_output/unique_drb_sites.json -o ../public/data/topojson/unique_drb_sites.json format=topojson precision=0.001

In [ ]:
# convert segment_geojson json in intermediate_output folder to topojson w/ reduced precision and save in topojson subfolder  of public data folder
! mapshaper -i intermediate_output/segment_data.json -o ../public/data/topojson/segment_data.json format=topojson precision=0.0001

In [ ]:
# convert NHDWaterbody_DelawareBay_pt6per_smooth.json in intermediate_output folder to topojson w/ reduced precision and save in topojson subfolder of public data folder
! mapshaper -i intermediate_output/NHDWaterbody_DelawareBay_pt6per_smooth.json -o ../public/data/topojson/DelawareBay.json format=topojson precision=0.001